In [1]:
import os

In [2]:
pwd

'/config/workspace/red_wine_quality/research'

In [3]:
os.chdir('../')

In [4]:
pwd

'/config/workspace/red_wine_quality'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str


In [6]:
from wine_quality.utils.common import read_yaml, create_directories, save_json
from wine_quality.constants import *

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, schema_filepath=SCHEME_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
    
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params= params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config

In [7]:
import joblib
import os
from wine_quality import logger
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        predicted_qualities = model.predict(test_x)

        (rmse, mae, r2) = self.eval_metric(test_y, predicted_qualities)

        ## saving metrics as local
        scores = {"rmse": rmse, "mae":mae, "r2_score":r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)




In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-01-05 09:11:35,482: INFO: common: yaml file: config/config.yaml loaded successfully.]
[2024-01-05 09:11:35,486: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-01-05 09:11:35,488: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-01-05 09:11:35,489: INFO: common: created directory at: artifacts]
[2024-01-05 09:11:35,490: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-05 09:11:35,502: INFO: common: json file save at: artifacts/model_evaluation/metrics.json]
